In [110]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from shapely.geometry import shape
import rasterio.features
from matplotlib import pyplot as plt
from shapely.geometry import Polygon, Point

crs = "EPSG:2232"

In [ ]:
# Color
%run ../assets/colors.py
# rcParams for MPL
%run ../assets/rcParams.py

In [134]:
land_cover_2009 = rio.open("../data/land-cover/2009.tif")
land_cover_2019 = rio.open("../data/land-cover/2019.tif")

Function to transform land cover rasters to fishnet or points.

In [138]:
def raster_to_shape(raster, crs, to_point=False, value_column="value"):
    data_array = raster.read(1)
    transform = raster.transform
    shapes_data = []
    height, width = data_array.shape

    if to_point:
        for row in range(height):
            for col in range(width):
                value = data_array[row, col]
                if np.isnan(value):
                    continue
                x, y = transform * (col + 0.5, row + 0.5)
                # Create a polygon for the cell
                point = Point(x, y)
                # Add the polygon and its value to the list
                shapes_data.append({"geometry": point, value_column: value})
        return gpd.GeoDataFrame(shapes_data, crs=crs)
    
    for row in range(height):
        for col in range(width):
            value = data_array[row, col]
            if np.isnan(value):
                continue
            x_min, y_max = transform * (col, row)
            x_max, y_min = transform * (col + 1, row + 1)
            # Create a polygon for the cell
            polygon = Polygon([(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)])
            # Add the polygon and its value to the list
            shapes_data.append({"geometry": polygon, value_column: value})
    return gpd.GeoDataFrame(shapes_data, crs=crs)

Get a fishnet containing two columns, `land_cover_2009` and `land_cover_2019`

In [158]:
fishnet = (
    raster_to_shape(
        land_cover_2009, crs, to_point=False, value_column="land_cover_2009"
    )
    .query("land_cover_2009 != 0")
    .copy()
)
points_2019 = (
    raster_to_shape(land_cover_2019, crs, to_point=True, value_column="land_cover_2019")
    .query("land_cover_2019 != 0")
    .copy()
)

# Join points to fishnet, no index right
fishnet = gpd.sjoin(fishnet, points_2019, how="left", predicate="contains").drop(
    columns=["index_right"]
)

[Land cover data dictionary](https://www.mrlc.gov/data/legends/national-land-cover-database-class-legend-and-description)
Make two columns of boolean values: whether developed in the years 2009 or 2019

In [162]:
developed = [21, 22, 23, 24]
for year in [2009, 2019]:
    fishnet[f"developed_{year}"] = fishnet[f"land_cover_{year}"].isin(developed)
    fishnet.drop(columns=[f"land_cover_{year}"], inplace=True)

In [168]:
import cenpy

In [224]:
counties = ["001", "005", "031", "035", "014", "039", "059", "093", "019", "047"]

In [275]:
datasets = {
    "2010": {
        "dataset": "DECENNIALPL2010",
        "variable": "P001001",
        "geo_column": "GEOID10",
    }, 
    "2020": {
        "dataset": "DECENNIALPL2020",
        "variable": "P1_001N",
        "geo_column": "GEOID20",
    }
}

In [ ]:
from pygris import blocks

In [296]:
def get_data_with_geo(connection, county, year, geo_column, variable):
    # Get data from the Census API
    query = connection.query(
        cols=[variable],
        geo_unit="block:*",
        geo_filter={"state": "08", "county": county},
    )
    query[geo_column] = query.state + query.county + query.tract + query.block
    query = query.rename(columns={variable: "population"}).drop(
        columns=["state", "county", "tract", "block"]
    )

    # Get the block geometries
    blocks_gdf = blocks(
        state="08",
        county=county,
        year=int(year),
    )[[geo_column, "geometry"]]

    # Merge the data with the geometries
    query = blocks_gdf.merge(query, on=geo_column, how="left")
    return query.rename(
        columns={geo_column: "GEOID", "population": f"population_{year}"}
    )

In [291]:
test = get_data_with_geo(
    cenpy.remote.APIConnection(datasets["2020"]["dataset"]),
    "031",
    "2020",
    "GEOID20",
    "P1_001N",
)

In [294]:
test2 = get_data_with_geo(
    cenpy.remote.APIConnection(datasets["2010"]["dataset"]),
    "031",
    "2010",
    "GEOID10",
    "P001001",
)

In [297]:
# Create an empty DataFrame to store the results
population_2010 = pd.DataFrame()
population_2020 = pd.DataFrame()

In [298]:
# Loop through the census datasets and county FIPS codes to get block-level population data
for year, dataset in datasets.items():
    connection = cenpy.remote.APIConnection(dataset["dataset"])
    for county in counties:
        subset = get_data_with_geo(
            connection=connection,
            county=county,
            year=year,
            geo_column=dataset["geo_column"],
            variable=dataset["variable"],
        )
        globals()[f"population_{year}"] = pd.concat(
            [globals()[f"population_{year}"], subset], ignore_index=True
        )
        print(f"finished for {county} in {year}")

finished for 001 in 2010
finished for 005 in 2010
finished for 031 in 2010
finished for 035 in 2010
finished for 014 in 2010
finished for 039 in 2010
finished for 059 in 2010
finished for 093 in 2010
finished for 019 in 2010
finished for 047 in 2010
finished for 001 in 2020
finished for 005 in 2020
finished for 031 in 2020
finished for 035 in 2020
finished for 014 in 2020
finished for 039 in 2020
finished for 059 in 2020
finished for 093 in 2020
finished for 019 in 2020
finished for 047 in 2020


In [302]:
population_2010.plot(
    column="population_2010",
    legend=True,
    cmap="viridis",
)

<Axes: >